# Pipeline & Test
____________________________

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
import resreg
import itertools
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error

In [2]:
def temporal_aware_train_test_split(df, year_predict, provide_X_y=False):
    # Train test split
    df_train = df[df['year'] < year_predict].copy()
    df_test = df[df['year'] == year_predict].copy()
    
    if provide_X_y:
        # Obtain X_train, X_test, y_train, y_test
        y_train = df_train['Productivity']
        y_test = df_test['Productivity']

        X_train = df_train.drop(columns=['市町村名', 'Productivity', 'year'])
        X_test = df_test.drop(columns=['市町村名', 'Productivity', 'year'])
        return X_train, X_test, y_train, y_test
    else:
        return df_train, df_test

In [3]:
def trimmed_add_city_historical_mean_feature(df_train, df_test, year_predict):
    """
    Adds a feature representing the historical trimmed mean productivity per city
    for a given prediction year, avoiding lookahead bias.

    For each city, the minimum and maximum productivity values are excluded 
    before calculating the mean.

    Returns:
        tuple (pd.DataFrame, pd.DataFrame): Modified training and test DataFrames with the new feature.
    """

    # Create dictionary: trimmed mean productivity per city (excluding min and max)
    mean_yield_by_city = {}
    for city, group in df_train.groupby('市町村名')['Productivity']:
        if len(group) > 2:
            trimmed_group = group.drop([group.idxmin(), group.idxmax()])
            mean_yield_by_city[city] = trimmed_group.mean()
        else:
            # If there are only 2 or fewer values, just take the regular mean
            mean_yield_by_city[city] = group.mean()

    # Assign the trimmed mean values to training set
    df_train['trimmed_mean_productivity_city'] = df_train['市町村名'].map(mean_yield_by_city)

    # Apply the same trimmed mean values to test set
    df_test['trimmed_mean_productivity_city'] = df_test['市町村名'].map(mean_yield_by_city)
    
    # Separate features and target
    y_train = df_train['Productivity']
    y_test = df_test['Productivity']

    X_train = df_train.drop(columns=['市町村名', 'Productivity', 'year'])
    X_test = df_test.drop(columns=['市町村名', 'Productivity', 'year'])
    
    return X_train, X_test, y_train, y_test

In [4]:
def add_yearly_averages(df, features_to_average):
    """
    Computes the year average for selected features and adds them
    as new features with suffix '_year_average'.

    Parameters:
        df (pd.DataFrame): Original DataFrame with a 'year' column.
        features_to_average (list): List of feature names (str) to compute year averages.

    Returns:
        pd.DataFrame: DataFrame with new '_year_average' features added.
    """
    # Compute mean per year only for selected features
    yearly_means = df.groupby('year')[features_to_average].mean()
    yearly_means = yearly_means.add_prefix('year_average_').reset_index()

    # Merge back into the original DataFrame
    df = df.merge(yearly_means, on='year', how='left')

    return df

In [5]:
def LGBMR_model(X_train, X_test, y_train, y_test):
    """
    Quickly evaluates the performance of a model after preprocessing,
    using a balanced LightGBM configuration.
    """
    model = LGBMRegressor(
        objective='regression',
        metric='rmse',
        num_leaves=31,
        max_depth=4,
        min_child_samples=30,
        subsample=0.6,
        colsample_bytree=0.8,
        learning_rate=0.10,
        n_estimators=100,
        reg_alpha=0.0,
        reg_lambda=0.0,
        random_state=42,
        verbosity=-1
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    print(f"✅ Preprocessing Assessment — RMSE: {rmse:.4f}")
    
    return model

In [6]:
features_less_important = ["市町村コード", "NDVI_li_9", "NDVI_li_10",
                           "ET0_sum_veg", "ET0_sum_rep", "PTI_sum_veg", "PTI_sum_rep",
                           "SWD_sum_veg", "SWD_sum_rep", "SWD_mean_rep", "SWD_mean_veg",
                           "T2M_days_ideal_vegetative", "T2M_days_ideal_reproductive",
                           "T2M_days_low_vegetative", "T2M_days_low_reproductive",
                           "T2M_days_extremely_low_vegetative", "T2M_days_extremely_low_reproductive",
                           "ALLSKY_days_ideal_veg", "ALLSKY_days_ideal_rep", "ALLSKY_days_low_veg",
                           "ALLSKY_days_low_rep", "ALLSKY_days_high_veg", "ALLSKY_days_high_rep",
                           "RH2M_days_ideal_veg", "RH2M_days_high_veg", "RH2M_days_very_high_veg",
                           "RH2M_days_very_high_veg", "NDVI_li_5", "NDVI_li_6", "NDVI_li_7", "NDVI_li_8"]

In [7]:
key_features = ['GDD_sum_veg', 'GDD_sum_rep',
               'PTU_sum_veg', 'PTU_sum_rep',
               'VPD_sum_veg', 'VPD_sum_rep',
               'ALLSKY_NDVI_5', 'ALLSKY_NDVI_6',
               'ALLSKY_NDVI_7', 'ALLSKY_NDVI_8']

In [8]:
df = pd.read_csv('../datasets/weather_NDVI_harvest.csv').copy()

ndvi_cols = ['NDVI_li_5', 'NDVI_li_6', 'NDVI_li_7', 'NDVI_li_8']
months_list = [5, 6, 7, 8]
# ALLSKY x NDVI each month
for month in months_list:
    df[f'ALLSKY_NDVI_{month}'] = df[f'ALLSKY_{month}'] * df[f'NDVI_li_{month}']

df = df.drop(columns=features_less_important)
df = add_yearly_averages(df=df, features_to_average=key_features)

In [9]:
def add_city_historical_mean_feature(df_train, df_test, year_predict):
    """
    Adds a feature representing the historical mean productivity per city
    for a given prediction year, avoiding lookahead bias.

    Parameters:
        df (pd.DataFrame): Input DataFrame containing at least city, year, and target columns.
        year_predict (int): The year to isolate as validation/test. Data before this year is used to compute means.
        city_col (str): Name of the column indicating city/region.
        target_col (str): Name of the column containing the productivity target.
        fillna_global_mean (bool): Whether to fill missing cities in test set with global mean from training.

    Returns:
        tuple (pd.DataFrame, pd.DataFrame): Modified training and test DataFrames with the new feature.
    """

    # Create a dictionary with the mean yield per city (no lookahead!)
    mean_yield_by_city = df_train.groupby('市町村名')['Productivity'].mean().to_dict()

    # Assign the mean values directly to X_train, like a fit_transform
    df_train['mean_productivity_city'] = df_train['市町村名'].map(mean_yield_by_city)

    # Apply the same mean values to the corresponding cities in X_test, like a transform
    df_test['mean_productivity_city'] = df_test['市町村名'].map(mean_yield_by_city)
    
    # Obtain X_train, X_test, y_train, y_test
    y_train = df_train['Productivity']
    y_test = df_test['Productivity']

    X_train = df_train.drop(columns=['市町村名', 'Productivity', 'year'])
    X_test = df_test.drop(columns=['市町村名', 'Productivity', 'year'])
    
    return X_train, X_test, y_train, y_test

### 2022 test:
___________________

In [10]:
# --------- Configuration ---------
year_predict = 2022

# --------- Prepare empty results ---------
results = []

# --------- Model 1: LGBMR_model with trimmed_mean_productivity_city ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = trimmed_add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

model_2022 = LGBMR_model(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
y_pred_model = model_2022.predict(X_test)

results.append({
    "Model": "LGBMR_model (trimmed mean)",
    "RMSE": mean_squared_error(y_test, y_pred_model, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_model),
    "R²": r2_score(y_test, y_pred_model),
    "MedianAE": median_absolute_error(y_test, y_pred_model),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_model) / y_test)) * 100
})


# --------- Model 2: Trimmed mean baseline ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = trimmed_add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

y_pred_trimmed_baseline = X_test['trimmed_mean_productivity_city']

results.append({
    "Model": "Baseline (trimmed mean)",
    "RMSE": mean_squared_error(y_test, y_pred_trimmed_baseline, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_trimmed_baseline),
    "R²": r2_score(y_test, y_pred_trimmed_baseline),
    "MedianAE": median_absolute_error(y_test, y_pred_trimmed_baseline),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_trimmed_baseline) / y_test)) * 100
})


# --------- Model 3: Non-trimmed mean baseline ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

y_pred_mean_baseline = X_test['mean_productivity_city']

results.append({
    "Model": "Baseline (mean)",
    "RMSE": mean_squared_error(y_test, y_pred_mean_baseline, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_mean_baseline),
    "R²": r2_score(y_test, y_pred_mean_baseline),
    "MedianAE": median_absolute_error(y_test, y_pred_mean_baseline),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_mean_baseline) / y_test)) * 100
})


# --------- Show Results ---------
df_results = pd.DataFrame(results)
print("📊 Model Performance Comparison:\n")
print(df_results.to_markdown(index=False, floatfmt=".4f"))

✅ Preprocessing Assessment — RMSE: 26.1345
📊 Model Performance Comparison:

| Model                      |    RMSE |     MAE |     R² |   MedianAE |   MAPE (%) |
|:---------------------------|--------:|--------:|-------:|-----------:|-----------:|
| LGBMR_model (trimmed mean) | 26.1345 | 19.9692 | 0.7781 |    17.1617 |     3.5375 |
| Baseline (trimmed mean)    | 28.7381 | 21.8830 | 0.7317 |    19.7500 |     3.8525 |
| Baseline (mean)            | 32.0963 | 25.7849 | 0.6653 |    23.8125 |     4.5316 |


### 2023 test:
________________

In [11]:
# --------- Configuration ---------
year_predict = 2023

# --------- Prepare empty results ---------
results = []

# --------- Model 1: LGBMR_model with trimmed_mean_productivity_city ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = trimmed_add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

model_2022 = LGBMR_model(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
y_pred_model = model_2022.predict(X_test)

results.append({
    "Model": "LGBMR_model (trimmed mean)",
    "RMSE": mean_squared_error(y_test, y_pred_model, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_model),
    "R²": r2_score(y_test, y_pred_model),
    "MedianAE": median_absolute_error(y_test, y_pred_model),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_model) / y_test)) * 100
})


# --------- Model 2: Trimmed mean baseline ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = trimmed_add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

y_pred_trimmed_baseline = X_test['trimmed_mean_productivity_city']

results.append({
    "Model": "Baseline (trimmed mean)",
    "RMSE": mean_squared_error(y_test, y_pred_trimmed_baseline, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_trimmed_baseline),
    "R²": r2_score(y_test, y_pred_trimmed_baseline),
    "MedianAE": median_absolute_error(y_test, y_pred_trimmed_baseline),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_trimmed_baseline) / y_test)) * 100
})


# --------- Model 3: Non-trimmed mean baseline ---------
df_train, df_test = temporal_aware_train_test_split(df=df, year_predict=year_predict)

X_train, X_test, y_train, y_test = add_city_historical_mean_feature(
    df_train=df_train, df_test=df_test, year_predict=year_predict
)

y_pred_mean_baseline = X_test['mean_productivity_city']

results.append({
    "Model": "Baseline (mean)",
    "RMSE": mean_squared_error(y_test, y_pred_mean_baseline, squared=False),
    "MAE": mean_absolute_error(y_test, y_pred_mean_baseline),
    "R²": r2_score(y_test, y_pred_mean_baseline),
    "MedianAE": median_absolute_error(y_test, y_pred_mean_baseline),
    "MAPE (%)": np.mean(np.abs((y_test - y_pred_mean_baseline) / y_test)) * 100
})


# --------- Show Results ---------
df_results = pd.DataFrame(results)
print("📊 Model Performance Comparison:\n")
print(df_results.to_markdown(index=False, floatfmt=".4f"))

✅ Preprocessing Assessment — RMSE: 18.1711
📊 Model Performance Comparison:

| Model                      |    RMSE |     MAE |     R² |   MedianAE |   MAPE (%) |
|:---------------------------|--------:|--------:|-------:|-----------:|-----------:|
| LGBMR_model (trimmed mean) | 18.1711 | 12.3202 | 0.8697 |     7.5117 |     2.3298 |
| Baseline (trimmed mean)    | 17.8918 | 13.3606 | 0.8737 |    10.7143 |     2.4636 |
| Baseline (mean)            | 20.0963 | 16.3258 | 0.8407 |    14.4444 |     2.9839 |


In [12]:
X_train.columns

Index(['作付面積', 'PTU_sum_veg', 'PTU_sum_rep', 'VPD_sum_veg', 'VPD_sum_rep',
       'GDD_sum_veg', 'GDD_sum_rep', 'stress_combo_days_veg',
       'stress_combo_days_rep', 'ideal_combo_days_veg', 'ideal_combo_days_rep',
       'ALLSKY_days_extremely_low_veg', 'ALLSKY_days_extremely_low_rep',
       'ALLSKY_5', 'ALLSKY_6', 'ALLSKY_7', 'ALLSKY_8', 'RH2M_days_ideal_rep',
       'RH2M_days_high_rep', 'RH2M_days_very_high_rep', 'T2M_5', 'T2M_6',
       'T2M_7', 'T2M_8', '緯度', '経度', 'ALLSKY_NDVI_5', 'ALLSKY_NDVI_6',
       'ALLSKY_NDVI_7', 'ALLSKY_NDVI_8', 'year_average_GDD_sum_veg',
       'year_average_GDD_sum_rep', 'year_average_PTU_sum_veg',
       'year_average_PTU_sum_rep', 'year_average_VPD_sum_veg',
       'year_average_VPD_sum_rep', 'year_average_ALLSKY_NDVI_5',
       'year_average_ALLSKY_NDVI_6', 'year_average_ALLSKY_NDVI_7',
       'year_average_ALLSKY_NDVI_8', 'mean_productivity_city'],
      dtype='object')